<a href="https://colab.research.google.com/github/Vyoma-garg/Deep_Learning/blob/main/Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn  #all the NN network, CNN, Modules, loss functions, activation fns
import torch.optim as optim  #SGD, ADAM
import torch.nn.functional as F  #activation fn, 
from torch.utils.data import DataLoader  #easier data managemnet, creates mini batches of the data
import torchvision.datasets as datasets   #standard datasets MNISt ,etc
import torchvision.transforms as transforms  # transformations 

Create Fully Connected Network

In [12]:
class CNN(nn.Module): #inherit from nn.Module class
    def __init__(self, in_channels=1, num_classes=10):  #1 channel for black and white images
        super(CNN,self).__init__()   
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # to get the same input ouput 
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))  #14x14
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1)) #14x14
        self.fc1 = nn.Linear(16*7*7,num_classes)

    def forward(self, x):    
        x = F.relu(self.conv1(x)) 
        x=self.pool(x) 
        x = F.relu(self.conv2(x))
        x=self.pool(x)
        #print(x.shape)
        x=x.reshape(x.shape[0],-1) 
        x = self.fc1(x)
        return x



Checking 

In [ ]:
#model = CNN()
#x = torch.randn(64,1,28,28)  #randomly generated data, batch size=4, number of examples used simultaneously
#print(x)
#print('\n',model(x).shape,'\n')  #64x10   
#print(model(x)) 

Setting device

In [14]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameters

In [15]:
in_channels=1
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=1

Loading dataset

In [16]:
train_dataset = datasets.MNIST(root = '/content', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = '/content', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)


Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw




Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


Initialize network

In [17]:
model=CNN().to(device)

Loss and Optimizer

In [18]:
criterion= nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

Train Network

In [19]:
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):  #enumerate helps in getting the batch index of the data 
     #Get data to cuda
      data = data.to(device)
      targets = targets.to(device)

      #print(data.shape)  #([64,1,28,28])  num of examples, number of channels(black white img), height width of each image
                        
      #get correct shape
      #data=data.reshape(data.shape[0],-1)  #flatten to single dim: ([64,784])
      #print(data.shape) 

      #forward
      scores=model(data)
      loss=criterion(scores, targets) 

      #backward
      optimizer.zero_grad() #
      loss.backward()

      #gradient descent or adam step
      optimizer.step() #update weights based on the gradients



Check accuracy

In [20]:
def check_accuracy(loader,model):
  if loader.dataset.train:
        print('Checking Accuracy on Training Data')
  else:
        print('Checking Accuracy on Test Data')
  num_correct=0
  num_samples=0
  model.eval()

  with torch.no_grad():  #while checking accuracy no need to calculate the gradients again
    for x,y in loader:
      x=x.to(device=device)
      #x=x.reshape(x.shape[0],-1)

      scores=model(x)  #64x10 shape of the scores, which is max of those 10 digits
      values,predictions =scores.max(1)  #interested in index of the max value
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)  #first dimension that is 64 x 10==64
    print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

  model.train()
  

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 57691/ 60000 with accuracy 96.15 

Checking Accuracy on Test Data
Got 9642/ 10000 with accuracy 96.42 

